<a href="https://colab.research.google.com/github/shub2022/Text_classifier_Healthcare/blob/main/Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from fastai.text.all import *

from google.colab import drive
import os

drive.mount('/content/drive')
path = '/content/drive/MyDrive/Kaggle_int_hc'
os.listdir(path)


Mounted at /content/drive


['train', 'valid']

In [44]:
# DataBlock is an API which is used for building a dataloader

# https://docs.fast.ai/tutorial.datablock.html

dblock = DataBlock(blocks=(TextBlock.from_folder(path=path), CategoryBlock),
                   get_items=partial(get_text_files, folders=['train', 'valid']),
                   get_y=parent_label,
                   splitter=GrandparentSplitter(valid_name='valid'))

In [34]:
# Now creating a dataloader using dblock created above

dls = dblock.dataloaders(source=path, bs=32)

In [38]:
len(dls.train), len(dls.valid), len(dls.train.dataset), len(dls.valid.dataset)

(84, 22, 2707, 677)

In [39]:
dls.show_batch()

,text,category
0,"xxbos xxmaj chiropractic xxup ime with old files review . xxmaj detailed xxmaj thoracic xxmaj spine xxmaj examination . transcription : xxup date xxup of xxup injury : xxmaj october 4 , 2 xxrep 3 0 , date xxup of xxup examination : xxmaj september 5 , 2003,examining xxup physician : x xxup y , md , prior to the beginning of the examination , it is explained to the examinee that this examination is intended for evaluative purposes only , and that it is not intended to constitute a general medical examination . xxmaj it is explained to the examinee that the traditional doctor - patient relationship does not apply to this examination , and that a written report will be provided to the agency requesting this examination . xxmaj it has also been emphasized to the examinee that he should not attempt any physical activity beyond his tolerance",IME-QME-Work Comp etc.
1,"xxbos xxmaj chiropractic xxup ime with old files review . xxmaj detailed xxmaj thoracic xxmaj spine xxmaj examination . transcription : xxup date xxup of xxup injury : xxmaj october 4 , 2 xxrep 3 0 , date xxup of xxup examination : xxmaj september 5 , 2003,examining xxup physician : x xxup y , md , prior to the beginning of the examination , it is explained to the examinee that this examination is intended for evaluative purposes only , and that it is not intended to constitute a general medical examination . xxmaj it is explained to the examinee that the traditional doctor - patient relationship does not apply to this examination , and that a written report will be provided to the agency requesting this examination . xxmaj it has also been emphasized to the examinee that he should not attempt any physical activity beyond his tolerance",Chiropractic
2,"xxbos xxmaj pain management for post - laminectomy low back syndrome and radiculopathy . transcription : xxmaj mr . xxup xyz forgot his hearing aids at home today and is severely hearing impaired and most of the interview had to be conducted with me xxunk at him at the top of my voice . xxmaj for all these reasons , this was not really under the best circumstances and i had to xxunk the amount of time i spent trying to get a history because of the physical effort required in xxunk information from this patient . xxmaj the patient was seen late because he had not filled in the patient questionnaire . xxmaj to xxunk the history here , xxmaj mr . xxup xyz who is not very clear on events from the past , sustained a work - related injury some time in 1998 . xxmaj at that",Chiropractic
3,"xxbos xxmaj the patient is with multiple neurologic and nonneurologic symptoms including numbness , gait instability , decreased dexterity of his arms and general fatigue . xxmaj his neurological examination is notable for sensory loss in a length - dependent fashion in his feet and legs with scant fasciculations in his calves . transcription : xxup history xxup of xxup present xxup illness : , xxmaj the patient is a 63 - year - old left - handed gentleman who presents for further evaluation of multiple neurological symptoms . i asked him to discuss each symptom individually as he had a very hard time describing the nature of his problems . xxmaj he first mentioned that he has neck pain . xxmaj he states that he has had this for at least 15 years . xxmaj it is worse with movement . xxmaj it has progressed very slowly over the",Consult - History and Phy.
4,"xxbos xxmaj autopsy of a white female who died of acute combined drug intoxication . transcription : xxup clothing : , xxmaj the body is xxunk in a light green hospital gown , which is intact , dry and clean . xxmaj there is no xxunk present . ,external xxup examination : , xxmaj the body is that of a well - developed , well - nourished white woman appearing the offered age of 39 years . xxmaj the body measures 71 inches and weighs 178 pounds . ,the xxunk body is well preserved and cool to touch due to xxunk . xxmaj xxunk mortis is fully developed in the major muscle group

In [40]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [41]:
# Fine tuning the pre-trained AWD LSTM model using the training data

learn.fine_tune(4, 1e-2)


epoch,train_loss,valid_loss,accuracy,time
0,2.556282,2.154324,0.347120,02:13


epoch,train_loss,valid_loss,accuracy,time
0,2.013224,2.011767,0.335303,01:28
1,1.887022,1.908812,0.323486,01:28
2,1.679380,1.762743,0.317578,01:28
3,1.478179,1.757976,0.302806,01:28


Performance is pretty bad

Let us build an encoder which will be specific for this task and will then fine tune again

In [52]:
files = get_text_files(path)

# Create a simple DataFrame
df = pd.DataFrame({'text': files})

dblock_lm = DataBlock(blocks=(TextBlock.from_folder(path=path, is_lm=True)),
                    get_items=(get_text_files),
                    get_x=ColReader('text'),
                    splitter=RandomSplitter(valid_pct=0.1)  # Adjust valid_pct as needed
)


In [51]:
dls_lm = dblock_lm.dataloaders(source=path, bs=64, seq_len=72)

TypeError: 'PosixPath' object is not subscriptable

In [54]:
df.tail()

,text
3379,/content/drive/MyDrive/Kaggle_int_hc/valid/ Dermatology/556.txt
3380,/content/drive/MyDrive/Kaggle_int_hc/valid/ Dermatology/597.txt
3381,/content/drive/MyDrive/Kaggle_int_hc/valid/ Cosmetic - Plastic Surgery/474.txt
3382,/content/drive/MyDrive/Kaggle_int_hc/valid/ Cosmetic - Plastic Surgery/498.txt
3383,/content/drive/MyDrive/Kaggle_int_hc/valid/ Physical Medicine - Rehab/631.txt
